# Project Overview

In this project, we will build a market simulator that accepts trading orders while monitoring the value and performance of the portfolio.

## Our Imports

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt

## Compute Portfolio Value

## Execute Orders

## Assess Portfolio Performance

## Implementing Transaction Costs

## Implementing Market Impact

## Randomizing Orders